## Molecular Dynamics Simulation of Heteropolymer Chromatin

This notebook implements a coarse-grained molecular dynamics simulation to study the 3D organization of chromatin (DNA-protein complexes) within the nuclear environment. The simulation uses OpenMM (chunkchromatin is a wrapper) as the underlying physics engine to model the dynamic behavior of chromatin fibers.

### Core Components

**System Setup:**
- **N = 100 monomers**: Represents a chromatin fiber segment with 100 coarse-grained beads aka monomers
- **Density = 0.33**: Sets the packing density within a cubic simulation box
- **Chains**: Defines two polymer chains (0-50, 50-100) representing chromatin domains, False indicates the chains are not circular
- **Monomer types**: This describes the type of each particle

**Force Field Implementation:**
- **Harmonic bonds**: Maintains polymer connectivity with spring-like forces
- **Angle forces**: Preserves chromatin fiber stiffness and persistence length
- **Spherical confinement**: Models nuclear boundary constraints
- **Polynomial repulsive forces**: Prevents monomer overlap (excluded volume effects)
- **Non-bonded pair potentials**: Implements sequence-specific interactions via interaction matrix
- **Lamina interactions**: Pins C-type monomers and attracts B-type monomers to nuclear periphery

### Simulation Protocol

**Production Phase:**
- 100 blocks of 100,000 timesteps each (10 fs timestep)
- Uses variable Langevin integrator at 300K
- HDF5 trajectory output for analysis

### Physical Parameters

- **Temperature**: 300K (physiological conditions)
- **Damping**: γ = 0.05 ps⁻¹ (Langevin thermostat)
- **Interaction matrix**: 3×3 matrix defining monomer-type affinities
- **Box size**: Calculated from monomer count and target density

This simulation framework enables investigation of chromatin organization principles, compartmentalization dynamics, and the role of sequence-specific interactions in 3D genome architecture.


In [18]:
import argparse
import numpy as np
import os
from chunkchromatin.simulation import Simulation
from chunkchromatin.chromosome import Chromosome
from chunkchromatin.lamina import Lamina
from chunkchromatin.hdf5_format import HDF5Reporter
from chunkchromatin.simulation import EKExceedsError
import openmm as mm
from initial_conformations import create_random_walk

import json

In [19]:
import numpy as np

def generate_pattern_blocks():
    """
    Generate monomer_types array for 120 monomers with fixed block pattern:
    4 repetitions of {10*A, 10*B, 10*C}.
    A=0, B=1, C=2.
    """
    monomer_types = []
    for _ in range(4):
        monomer_types.extend([0]*10)  # A block
        monomer_types.extend([1]*10)  # B block
        monomer_types.extend([2]*10)  # C block
    return np.array(monomer_types, dtype=int)


# import numpy as np

# def generate_block_copolymer(N, min_block_size=5, types=(0, 1, 2), seed=None):
#     """
#     Generate a block copolymer array of length N, with each block at least min_block_size long,
#     and block types randomly alternating between the given types (no adjacent blocks have the same type).
#     """
#     if seed is not None:
#         np.random.seed(seed)
#     monomer_types = np.empty(N, dtype=int)
#     current = 0
#     last_type = None
#     while current < N:
#         # Choose a type different from the last block
#         possible_types = [t for t in types if t != last_type]
#         block_type = np.random.choice(possible_types)
#         # Choose block length (at least min_block_size, but not to exceed N)
#         max_block = N - current
#         block_len = np.random.randint(min_block_size, max_block + 1) if max_block > min_block_size else max_block
#         monomer_types[current:current+block_len] = block_type
#         current += block_len
#         last_type = block_type
#     return monomer_types


In [20]:
N = 120   # total monomers
density = 0.33
chains = [(0, 60, False), (60, 120, False)]   # two chains of 60 beads each

# Fixed pattern monomers
monomer_types = generate_pattern_blocks()
np.save("monomer_types_chr.npy", monomer_types)

# Quick check
print("Counts:", 
      "A:", np.sum(monomer_types == 0),
      "B:", np.sum(monomer_types == 1),
      "C:", np.sum(monomer_types == 2))

# Interaction matrix
interaction_matrix = np.array([
    [0.05, 0.05, 0.08],
    [0.05, 0.13, 0.17],
    [0.08, 0.17, 0.22]
])

# Initial positions
box_length = (N / density) ** (1 / 3.)
monomer_positions = create_random_walk(step_size=1, N=N)

# Output folder
out_dir = 'test_output_chr_1'
os.makedirs(out_dir, exist_ok=True)
reporter = HDF5Reporter(folder=out_dir, max_data_length=500, overwrite=True)

# Simulation setup
sim = Simulation(
    integrator_type="variableLangevin",
    temperature=300.0,  # K
    gamma=0.05,         # 1/ps
    timestep=5,         # fs
    platform='CPU',
    N=N,
    reporter=reporter
)

chromosome = Chromosome(N, chains, sim)
lamina = Lamina(N, chains, sim)

sim.set_positions(monomer_positions)

# Forces
harmonic_bond_force = chromosome.add_harmonic_bond()
angle_force = chromosome.add_angle_force()
nonbonded_pair_potential_force = chromosome.add_nonbonded_pair_potential(sim, interaction_matrix, monomer_types)
spherical_confinement_force = lamina.add_spherical_confinement(sim)

sim.add_force(harmonic_bond_force)
sim.add_force(angle_force)
sim.add_force(nonbonded_pair_potential_force)
sim.add_force(spherical_confinement_force)

# Finalize
sim.create_context()
sim.set_velocities()


############ this is the previous code ##########
# N = 100
# density = 0.33
# chains = [(0,50,False), (50,100,False)]

# #refactor - make this random initialization with same seed
# monomer_types = generate_block_copolymer(N, min_block_size=5, types=(0, 1, 2), seed=42)

# interaction_matrix = np.array([
#     [0.05, 0.05, 0.08],
#     [0.05, 0.13, 0.17],
#     [0.08, 0.17, 0.22]
# ])

# box_length = (N/density) ** (1/3.)
# monomer_positions = create_random_walk(step_size=1, N=N)

# out_dir = 'test_output'
# os.makedirs(out_dir, exist_ok=True)

# reporter = HDF5Reporter(folder=out_dir, max_data_length=500, overwrite=True)

# #make a different simulation object for equilibration
# sim = Simulation(
#     integrator_type="variableLangevin",
#     temperature=300.0,  # in Kelvin
#     gamma=0.05,         # in 1/ps
#     timestep=5,    # in fs
#     platform='CPU',
#     N=N,
#     reporter=reporter
# )

# chromosome = Chromosome(N, chains, sim)
# lamina = Lamina(N, chains,sim)

# sim.set_positions(monomer_positions)

# harmonic_bond_force = chromosome.add_harmonic_bond()
# angle_force = chromosome.add_angle_force()
# nonbonded_pair_potential_force = chromosome.add_nonbonded_pair_potential(sim,interaction_matrix,monomer_types)
# spherical_confinement_force = lamina.add_spherical_confinement(sim)

# sim.add_force(harmonic_bond_force)
# sim.add_force(angle_force)
# sim.add_force(nonbonded_pair_potential_force)
# sim.add_force(spherical_confinement_force)

# sim.create_context()
# sim.set_velocities()


Counts: A: 40 B: 40 C: 40


In [21]:
for _ in range(200):
    sim.run_simulation_block(1000)
    with open(f"{out_dir}/simulation_stats.txt", "a") as f:
        stats = str(sim.print_stats())
        f.write(stats + "\n")
    reporter.dump_data()

INFO:root:block    0 pos[1]=[-0.3 -1.5 -0.4] t=0.0ns kin=3.71 pot=3.92 SPS=3495



Simulation Statistics:
Current block: 1
Total steps: 1000
Simulation time: 7.89τ
Kinetic energy: 0.00 kT/particle
Potential energy: 0.00 kT/particle
Total energy: 0.01 kT/particle


INFO:root:block    1 pos[1]=[-0.8 4.5 -0.8] t=0.0ns kin=2.83 pot=3.15 SPS=3948



Simulation Statistics:
Current block: 2
Total steps: 2000
Simulation time: 15.80τ
Kinetic energy: 0.00 kT/particle
Potential energy: 0.00 kT/particle
Total energy: 0.01 kT/particle


INFO:root:block    2 pos[1]=[0.8 -1.2 0.0] t=0.0ns kin=2.34 pot=2.43 SPS=4256


...


Simulation Statistics:
Current block: 198
Total steps: 198000
Simulation time: 1910.07τ
Kinetic energy: 0.00 kT/particle
Potential energy: 0.00 kT/particle
Total energy: 0.00 kT/particle


INFO:root:block  198 pos[1]=[1.5 1.4 1.1] t=1.9ns kin=1.56 pot=1.77 SPS=1638



Simulation Statistics:
Current block: 199
Total steps: 199000
Simulation time: 1919.83τ
Kinetic energy: 0.00 kT/particle
Potential energy: 0.00 kT/particle
Total energy: 0.00 kT/particle


INFO:root:block  199 pos[1]=[0.3 0.4 4.3] t=1.9ns kin=1.68 pot=1.59 SPS=1699



Simulation Statistics:
Current block: 200
Total steps: 200000
Simulation time: 1929.90τ
Kinetic energy: 0.00 kT/particle
Potential energy: 0.00 kT/particle
Total energy: 0.00 kT/particle


In [25]:
import h5py

file_path = "C:/Users/arnav/Personal - Arnav Chhajed/Northwestern/genome_organization/examples/arnav/test_output_chr_1/blocks_1-1.h5"

with h5py.File(file_path, "r") as f:
    print("Top-level keys:", list(f.keys()))

    # Walk the file to see all groups/datasets
    def print_structure(name, obj):
        print(name, "->", type(obj))
    f.visititems(print_structure)



Top-level keys: ['1']
1 -> <class 'h5py._hl.group.Group'>
1/pos -> <class 'h5py._hl.dataset.Dataset'>
